In [63]:
def is_goal_state(state, goal_state):
    """Verifica se lo stato corrente è lo stato obiettivo."""
    return state == goal_state

def generate_state_code(state, step, positions):
    state_code = ""
    for pos in positions:
        if state[pos - 1] == 0:
            state_code += f"empty_pos{pos}_{step} &\n"
        else:
            tile_at_pos = state[pos - 1]
            state_code += f"tile{tile_at_pos}_pos{pos}_{step} &\n"
    return state_code

def generate_single_movement_constraint(t, moving_tiles, empty_pos, adjacent):
    """Genera un vincolo che assicura che solo una tessera si muova per step temporale."""
    constraints = []
    for i, tile in enumerate(moving_tiles):
        from_pos = adjacent[empty_pos + 1][i]
        other_moves = [
            f"mv_{t}_{moving_tiles[j]}_{adjacent[empty_pos + 1][j]}_{empty_pos + 1}" 
            for j in range(len(moving_tiles)) if j != i
        ]
        constraints.append(f"(mv_{t}_{tile}_{from_pos}_{empty_pos + 1} -> (!" + " & !".join(other_moves) + "))")

    return " & ".join(constraints) + " &\n" if constraints else ""

def generate_frame_axioms_for_state(state, t, adjacent):
    frame_axioms = ""
    empty_pos = state.index(0)
    moving_tiles = [state[adj - 1] for adj in adjacent[empty_pos + 1] if state[adj - 1] != 0]
    
    # Genera assiomi di frame
    for tile, from_pos in zip(moving_tiles, adjacent[empty_pos + 1]):
        # Assiomi per movimento tessera
        frame_axioms += f"(mv_{t}_{tile}_{from_pos}_{empty_pos + 1} -> (tile{tile}_pos{empty_pos + 1}_{t + 1} & empty_pos{from_pos}_{t + 1})) &\n"
        frame_axioms += f"(!mv_{t}_{tile}_{from_pos}_{empty_pos + 1} -> (tile{tile}_pos{from_pos}_{t} & !empty_pos{from_pos}_{t})) &\n"
    
    # Aggiungi il vincolo di un solo movimento per step
    single_movement_constraint = generate_single_movement_constraint(t, moving_tiles, empty_pos, adjacent)
    if single_movement_constraint:
        frame_axioms += single_movement_constraint

    return frame_axioms





def generate_frame_axioms(states, adjacent):
    """Genera gli assiomi di frame per tutti gli stati intermedi."""
    all_frame_axioms = "% Frame axioms\n"
    for t, state in enumerate(states):
        all_frame_axioms += generate_frame_axioms_for_state(state, t, adjacent)
    
    return all_frame_axioms

def generate_early_stopping_condition(goal_state, steps, positions):
    stopping_condition = "% Early stopping condition if goal state is reached\n"
    for t in range(1, steps):
        conditions_for_state = []
        for pos in positions:
            tile_at_pos = goal_state[pos - 1]
            if tile_at_pos != 0:
                conditions_for_state.append(f"tile{tile_at_pos}_pos{pos}_{t}")
        if conditions_for_state:
            stopping_condition += "(" + " & ".join(conditions_for_state) + " -> " + " & ".join([
                f"tile{goal_state[pos - 1]}_pos{pos}_{steps}" for pos in positions if goal_state[pos - 1] != 0
            ]) + ") &\n"
    return stopping_condition

def generate_next_states(state, adjacent):
    empty_pos = state.index(0) + 1  # Aggiungi 1 per allineare con le chiavi nel dizionario 'adjacent'
    next_states = []

    for adj_pos in adjacent[empty_pos]:
        if state[adj_pos - 1] != 0:
            new_state = state.copy()
            new_state[empty_pos - 1], new_state[adj_pos - 1] = new_state[adj_pos - 1], 0  # Scambio la tessera con lo spazio vuoto
            next_states.append(new_state)

    return next_states




def count_correct_tiles(state, goal_state):
    return sum(s == g for s, g in zip(state, goal_state))

def recursive_state_generation(current_states, adjacent, steps_remaining, goal_state, seen_states):
    if steps_remaining == 0:
        return list(seen_states)

    for state in current_states:
        if tuple(state) not in seen_states:
            seen_states.add(tuple(state))
            next_states = generate_next_states(state, adjacent)
            recursive_state_generation(next_states, adjacent, steps_remaining - 1, goal_state, seen_states)

    return list(seen_states)



def generate_limboole_code_2x2_final(steps, initial_state, goal_state):
    adjacent = {
        1: [2, 3],
        2: [1, 4],
        3: [1, 4],
        4: [2, 3]
    }

    positions = [1, 2, 3, 4]
    seen_states = set()  # Inizializza seen_states

    code = "% Stato iniziale\n" + generate_state_code(initial_state, 0, positions)
    all_intermediate_states = recursive_state_generation([initial_state], adjacent, steps, goal_state, seen_states)  # Aggiungi seen_states

    for t, state in enumerate(all_intermediate_states, 1):
        code += f"% Stato intermedio {t}\n" + generate_state_code(state, t, positions)

    code += "% Stato finale\n" + generate_state_code(goal_state, steps, positions)
    code += generate_frame_axioms(all_intermediate_states, adjacent)
    code += generate_early_stopping_condition(goal_state, steps, positions)

    return code.rstrip(" &\n")

# Test the function
steps = 3
initial_state = [0, 1, 3, 2]
# 0 1
# 3 2
goal_state = [1, 2, 3, 0]

limboole_code_2x2_final = generate_limboole_code_2x2_final(steps, initial_state, goal_state)
print(limboole_code_2x2_final)



% Stato iniziale
empty_pos1_0 &
tile1_pos2_0 &
tile3_pos3_0 &
tile2_pos4_0 &
% Stato intermedio 1
tile3_pos1_1 &
tile1_pos2_1 &
empty_pos3_1 &
tile2_pos4_1 &
% Stato intermedio 2
tile1_pos1_2 &
empty_pos2_2 &
tile3_pos3_2 &
tile2_pos4_2 &
% Stato intermedio 3
empty_pos1_3 &
tile1_pos2_3 &
tile3_pos3_3 &
tile2_pos4_3 &
% Stato intermedio 4
tile1_pos1_4 &
tile2_pos2_4 &
tile3_pos3_4 &
empty_pos4_4 &
% Stato intermedio 5
tile3_pos1_5 &
tile1_pos2_5 &
tile2_pos3_5 &
empty_pos4_5 &
% Stato finale
tile1_pos1_3 &
tile2_pos2_3 &
tile3_pos3_3 &
empty_pos4_3 &
% Frame axioms
(mv_0_3_1_3 -> (tile3_pos3_1 & empty_pos1_1)) &
(!mv_0_3_1_3 -> (tile3_pos1_0 & !empty_pos1_0)) &
(mv_0_2_4_3 -> (tile2_pos3_1 & empty_pos4_1)) &
(!mv_0_2_4_3 -> (tile2_pos4_0 & !empty_pos4_0)) &
(mv_0_3_1_3 -> (!mv_0_2_4_3)) & (mv_0_2_4_3 -> (!mv_0_3_1_3)) &
(mv_1_1_1_2 -> (tile1_pos2_2 & empty_pos1_2)) &
(!mv_1_1_1_2 -> (tile1_pos1_1 & !empty_pos1_1)) &
(mv_1_2_4_2 -> (tile2_pos2_2 & empty_pos4_2)) &
(!mv_1_2_4_2 -> (tile2

In [64]:
import subprocess

def run_limboole_with_exe(input_string, exe_path="limbooleAPE.exe"):

    try:
        # Esegui Limboole utilizzando il file .exe
        process = subprocess.run(
            [exe_path, "-s"],
            input=input_string,
            text=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        # Restituisco l'output o l'errore
        if process.stderr:
            return f"Errore: {process.stderr}"
        else:
            return process.stdout
    except subprocess.SubprocessError as e:
        return f"Errore nell'esecuzione: {e}"

# Esegui Limboole e ottieni i risultati
result = run_limboole_with_exe(limboole_code_2x2_final)
print(result)


% SATISFIABLE formula (satisfying assignment follows)
empty_pos1_0 = 1
tile1_pos2_0 = 1
tile3_pos3_0 = 1
tile2_pos4_0 = 1
tile3_pos1_1 = 1
tile1_pos2_1 = 1
empty_pos3_1 = 1
tile2_pos4_1 = 1
tile1_pos1_2 = 1
empty_pos2_2 = 1
tile3_pos3_2 = 1
tile2_pos4_2 = 1
empty_pos1_3 = 1
tile1_pos2_3 = 1
tile3_pos3_3 = 1
tile2_pos4_3 = 1
tile1_pos1_4 = 1
tile2_pos2_4 = 1
tile3_pos3_4 = 1
empty_pos4_4 = 1
tile3_pos1_5 = 1
tile1_pos2_5 = 1
tile2_pos3_5 = 1
empty_pos4_5 = 1
tile1_pos1_3 = 1
tile2_pos2_3 = 1
empty_pos4_3 = 1
mv_0_3_1_3 = 1
tile3_pos3_1 = 1
empty_pos1_1 = 1
tile3_pos1_0 = 1
mv_0_2_4_3 = 0
tile2_pos3_1 = 1
empty_pos4_1 = 0
empty_pos4_0 = 0
mv_1_1_1_2 = 1
tile1_pos2_2 = 1
empty_pos1_2 = 1
tile1_pos1_1 = 1
mv_1_2_4_2 = 0
tile2_pos2_2 = 1
empty_pos4_2 = 1
mv_2_1_2_1 = 1
empty_pos2_3 = 1
mv_2_3_3_1 = 0
tile3_pos1_3 = 1
empty_pos3_3 = 0
empty_pos3_2 = 0
mv_3_2_2_4 = 1
tile2_pos4_4 = 1
empty_pos2_4 = 1
mv_3_3_3_4 = 0
tile3_pos4_4 = 1
empty_pos3_4 = 0
mv_4_1_2_4 = 1
tile1_pos4_5 = 1
empty_pos2_5

In [65]:
# Assicurati di usare la funzione `recursive_state_generation` che hai già scritto
initial_state = [0, 1, 3, 2]
goal_state = [1, 2, 3, 0]
steps = 4
adjacent = {1: [2, 3], 2: [1, 4], 3: [1, 4], 4: [2, 3]}
seen_states = set()

all_intermediate_states = recursive_state_generation([initial_state], adjacent, steps, goal_state, seen_states)

# Stampa tutti gli stati intermedi
for i, state in enumerate(seen_states):
    print(f"Stato intermedio {i+1}: {state}")
# Prosegui con la generazione del codice Limboole e la sua esecuzione se necessario


Stato intermedio 1: (3, 1, 0, 2)
Stato intermedio 2: (3, 0, 2, 1)
Stato intermedio 3: (1, 0, 3, 2)
Stato intermedio 4: (0, 1, 3, 2)
Stato intermedio 5: (1, 2, 0, 3)
Stato intermedio 6: (1, 2, 3, 0)
Stato intermedio 7: (3, 1, 2, 0)


In [66]:
def generate_single_movement_constraint(t, moving_tiles, empty_pos, adjacent):
    """Genera un vincolo che assicura che solo una tessera si muova per step temporale."""
    constraints = []
    for i, tile in enumerate(moving_tiles):
        from_pos = adjacent[empty_pos + 1][i]
        other_moves = [
            f"mv_{t}_{moving_tiles[j]}_{adjacent[empty_pos + 1][j]}_{empty_pos + 1}" 
            for j in range(len(moving_tiles)) if j != i
        ]
        constraints.append(f"(mv_{t}_{tile}_{from_pos}_{empty_pos + 1} -> (!" + " & !".join(other_moves) + "))")

    return " & ".join(constraints) + " &\n" if constraints else ""

# Mini test per la funzione generate_single_movement_constraint
adjacent = {
    1: [2, 3],
    2: [1, 4],
    3: [1, 4],
    4: [2, 3]
}

# Scenario di test: lo spazio vuoto si trova in posizione 1, le tessere che possono muoversi sono 2 e 3
empty_pos = 1
moving_tiles = [2, 3]
t = 3  # Step temporale per il test

constraints = generate_single_movement_constraint(t, moving_tiles, empty_pos, adjacent)
print("Vincoli generati:\n", constraints)


Vincoli generati:
 (mv_3_2_1_2 -> (!mv_3_3_4_2)) & (mv_3_3_4_2 -> (!mv_3_2_1_2)) &

